This example assumes that you have an [Ollama](https://github.com/ollama/ollama) model running (with `ollama serve`) on some host that is reachable (whether a `localhost` of your computer or some other host with a public IP).
As a predictor, we use [txai_omics_3](https://github.com/SermanVS/txai_omics_3).

In [1]:
from llamba.chatmodels.ollama import OllamaModel

In [2]:
chatbot = OllamaModel(url="http://127.0.0.1:11434/", endpoint="api/generate", model="llama3", num_threads=1, check_connection_timeout=15, request_timeout=15)
connection = chatbot.check_connection()
print(connection)

True


### Get a bioage model

In [3]:
from pathlib import Path
import os
import shap
import numpy as np
import pickle
import torch
from txai_omics_3.models.tabular.widedeep.ft_transformer import WDFTTransformerModel, FN_CHECKPOINT, FN_SHAP, TRAIN_DATA_PATH
from llamba_library.bioage_model import BioAgeModel

# Model
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=fn_model)
bioage_model = BioAgeModel(model=model)

# SHAP
fn_shap = FN_SHAP

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
with open(fn_shap, 'rb') as handle:
    shap_dict = pickle.load(handle)

h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\pytorch_lightning\utilities\migration\utils.py:55: The loaded checkpoint was produced with Lightning v2.4.0, which is newer than your current Lightning version: v2.1.4
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


### Load data

In [4]:
import pandas as pd

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

my_df = pd.DataFrame(my_data, index=[0])

# Model data

train_data = pd.read_excel(TRAIN_DATA_PATH)

### Initialize a connector

In [5]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)

### Specify bioage clock

In [6]:
connector.specify_clock("Immunoage")

### Run an analysis task

In [7]:
res = connector.analyze(data=my_df, shap_dict=shap_dict, top_n=3, analyze_feats=True, analyze_risks=True)
print(res['analysis'])

Your bioage is 79 and your aging acceleration is -11, which means you are ageing slower than normal.

PDGFA: 1005.84429
CD40LG: 1853.847406
CXCL9: 2599.629474
Here is some more information about your data. 

PDGFA stands for Platelet-Derived Growth Factor A. It's a protein involved in cell proliferation, differentiation, and migration.

According to ImmunoAge analysis using the epigenetic clock, an increased level of PDGFA is associated with accelerated biological aging. Higher expression levels of PDGFA have been linked to increased cellular senescence, oxidative stress, and cellular transformation.

CD40LG (CD154) is a protein that belongs to the tumor necrosis factor (TNF) receptor superfamily. In the context of epigenetic aging, CD40LG is one of the biomarkers used in the ImmuneAging clock algorithm.

An increased level of CD40LG in blood cells has been associated with accelerated telomere shortening and epigenetic aging. Elevated CD40LG levels may indicate a higher rate of cellula

In [8]:
recommendations = connector.produce_recommendations()
print(recommendations)

To normalize the results and lower the chance of disease occurrence based on the analysis, I recommend the following:

1. Lifestyle modifications:
	* Increase physical activity to slow down biological aging (-11 acceleration indicates you are aging slower than normal). Aim for at least 30 minutes of moderate-intensity exercise per day.
	* Maintain a healthy body mass index (BMI) through balanced diet and regular exercise.
	* Practice stress-reducing techniques, such as meditation or yoga, to mitigate the effects of accelerated biological aging.

2. Target high PDGFA levels:
	* Consider supplements that promote telomere length maintenance, such as alpha-lipoic acid, coenzyme Q10, or resveratrol.
	* Engage in activities that promote cellular senescence reversal, such as exercise-induced stem cell mobilization.
	* Maintain a healthy sleep schedule to reduce oxidative stress and inflammation.

3. Address elevated CD40LG levels:
	* Consult with your healthcare provider about potential immun